# **Sequence alignment**

In many biological questions, you may want to compare your sequence to those in many other species to gain insights. BLAST is an extremely powerful tool specialized for paired sequence alignment. In the first part of this lecture, we will see how to use the NCBI web interface to perform BLAST, how to download the result, and finally how to perform multiple sequence alignment using the MUSCLE software.

It will be followed by an *optional* lecture, showcasing how to automate all of these steps using Biopython's special classes.



---

## **Sequence alignment using BLAST and MUSCLE**

Let's pick a protein and do some alignment.

Visit [NCBI](https://www.ncbi.nlm.nih.gov/taxonomy) and search for a species and a sequene you want to analyze. (Skimming the [Taxonomy Project](https://www.ncbi.nlm.nih.gov/books/NBK21100/) is helpful.)

1. In the search box, you can enter whatever species you want.
   Or, you can browse. Follow the "Browser" link under "Taxonomy Tools". 
2. Once you land in a page of your species, familiarize yourself to the contents. Check out the **Taxonomy ID, Genetic code translation table, Mitochondrial genetic code translation table**, etc.
3. On the right, click, the number link of **nucleotide**. It will bring you to the list of nucleotide records related to the species.  
4. In the search box, you can add specific protein names or other keywords to narrow down the search result.
5. Click an entry you want to check. It will show a GenBank information of the entry.
6. At the top, you can find the **ID**, which is you want to use to retrieve from Biopython. Make a note of it.
7. On the right side, click **Run BLAST** under **Analyze this sequence**.
8. In the **BLAST** page, read options. For example, you can specify the organism you want to search.
9. Click the **BLAST** button at the bottom. If you don't see any matching sequence, go back to the BLAST page and loosen the similarity threshold under **Program Selection** to "More dissimilar sequences". ("Somewhat similar sequences" is too loose.)
10. In the result page, the similar sequences are typically sorted in the order of **E-value**. For more details, check out [How_to_read_result](https://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLAST_NewResultPage.pdf).
11. You can refine the result using **Filter Results** box on the top right.
12. There are serveral tabs you can explore. For example, **Alignments** tab shows alignment between your sequence and found sequences.
13. You can download the result in various standard formats. Try downloading **FASTA (complete sequence)**, **FASTA (aligned sequences)**, and **XML**. Check the difference of these files. Limit the number of sequences reasonably small (5-10) if you want to perform multiple sequence alignment, as described next.
14. Download **FASTA (complete sequence)**. Then run the following command in the terminal:
```
>> MUSCLE/muscle3.8.31_i86linux64 -in downloade_filename -out output_filename
```
15. Exam the output file to see the result of multiple alignment.

Note: BLAST is a paired sequence alignment algorithm. For multiple sequence (simultaneous) alignment, other algorithms are used, such as ClustalW (oldest), MAFFT, and MUSCLE. MUSCLE is the most recent algorithm and can be downloade at https://www.drive5.com/muscle/downloads.htm. MAFFT and MUSCLE are the most used algorithms to align multiple sequences.

REF: You can find more information about BLAST at https://www.youtube.com/playlist?list=PLH-TjWpFfWrtjzMCIvUe-YbrlIeFQlKMq




---

### *(OPTIONAL)*


## **Running BLAST over the internet**

- `qblast()`: calls the onilne BLAST and returns the result in various formats

<sub><sub>For more details, see **Chapter 7** of the official [Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html).</sub></sub>

In [ ]:
from Bio.Blast import NCBIWWW
#help(NCBIWWW.qblast)

In [ ]:
# "plastn": search method, "nt": database, the last argument is a query sequence ID
# The last option is used to limit our search in Homo Spaiens. ([ORGN] means that we 
# search "organism" with name Homo Sapiens.)
result_handle = NCBIWWW.qblast("blastn", "nt", "NM_001316525", entrez_query='homo sapiens[ORGN]')

# "plastn" can be very slow, not because the algorithm is slow, but because
# its search is quite forgiving and thus it searches a huge database.
# See https://ncbi.github.io/blast-cloud/dev/api.html for more options

In [ ]:
##### BE CAREFUL!!! "read()" function will discard the content after reading.
##### Therefore, the second reading will result in an empty string.
##### It is a good practice to save the result.

save_handle = open("NM_001316525_HS.xml", 'w')
save_handle.write(result_handle.read())

In [ ]:
# For qblast, we can directly use a sequence

from Bio.Seq import Seq
mystery_seq = Seq("ATGCGTTAAAGCGTTAGCTAGCTTAAAGCTAGTATCGACTGCATCGACTCATCATCCTA")

result_handle = NCBIWWW.qblast("blastn", "nt", mystery_seq)
save_handle = open("mystery_seq.xml", 'w')
save_handle.write(result_handle.read())

In [ ]:
# If we have a fasta file, that can be used without modification

from Bio.Blast import NCBIWWW
fasta_string = open("some.fasta").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
save_handle = open("some.xml", 'w')
save_handle.write(result_handle.read())

# Of if we have used SeqIO to read in the fasta file, we can pass the 'seq'
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("some.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)
save_handle = open("some.xml", 'w')
save_handle.write(result_handle.read())



## **Parsing the BLAST result**

In [ ]:
# Basic parsing

from Bio.Blast import NCBIXML

result_handle = open("NM_001316525.xml")
blast_record = NCBIXML.read(result_handle) 
result_handle.close()

In [ ]:
# use parse() if many results. This allows for-loop iteration
result_handle = open("NM_001316525.xml")
blast_record_iter = NCBIXML.parse(result_handle)
for blast_record in blast_record_iter:
    print(blast_record)

dir(blast_record)

In [ ]:
# OR, if you want to combine all results,
result_handle = open("NM_001316525.xml")
blast_record_iter = NCBIXML.parse(result_handle)
blast_records = list(blast_record_iter)
print(blast_records)
blast_record = blast_records[0]
print(blast_record)

In [ ]:
print('There are', len(blast_record.alignments), 'records')
blast_record.alignments[0]

In [ ]:
# 'Bio.Blast.Record.Blast' has alignments, which is list of hit sequences
# In each alignment, there are HSPs (High Scoring Pair). In BLAST, there are usually
# only 1. But in other search engines, such as BLAT, it can be many.
# Within each HSP, there is a value called "expect" that contains e-value.

# Use dir() to examine the properties

for alignment in blast_record.alignments[10:15]:
    print("    ")
    print("******************************************* Alignment ************************")
    print("Sequence:", alignment.title)
    print("Hit seq ID:", alignment.hit_id)
    print("Accession:", alignment.accession)
    print("Length:", alignment.length)
    
    for hsp in alignment.hsps: # High Scoring Pairs. Occasionally more than 1 for BLAST. (Other serach engines often return more than 1 HSPs for each alignment.)
        if hsp.expect < 10e-30: # This filters out low quality alignments.
                                # You should change this option for your research need.
            print("***** HSP *****")
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

# Now you can do whatever you wish to do. If you're interested, Biopython provides 
# more systematic way of parsing the result.

# **Automate the whole process**

### **Step 1: BLAST**

In [ ]:
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastn", "nt", "NM_001316525", entrez_query='homo sapiens[ORGN]')
save_handle = open("NM_001316525_HS.xml", 'w')
save_handle.write(result_handle.read())

### **Step 2: Parse**

In [ ]:
from Bio.Blast import NCBIXML
result_handle = open("NM_001316525_HS.xml")
blast_record = NCBIXML.read(result_handle) 
result_handle.close()

### **Step 3: Collect accession IDs**
This is an important step in real life situation. The criteria to select interesting sequence depend on the biological question. The criteria should be quantitative and should be programmed for objective results.

In [ ]:
# Here, we will simply pick the top 5.

top_ids = [ alignment.accession  for alignment in blast_record.alignments[:5]]
top_ids

### **Step 4: Download sequence data**

In [ ]:
# The following function, "retrieve_GenBank_info()" is from the last class
import os
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "sungsoo@ucsb.edu"  # Always tell NCBI who you are

In [ ]:
# This function retrieves GenBank record of the id.

def retrieve_GenBank_info(id):
    filename = id + ".gbk"

    print("Downloading GenBank information of", id, "...")
    net_handle = Entrez.efetch(
        db="nucleotide", id=id, rettype="gb", retmode="text"
    )
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()
    print("Saved")

    print("Parsing...")
    record = SeqIO.read(filename, "genbank")
    return record

In [ ]:
ids=top_ids
records = [ retrieve_GenBank_info(id)  for id in ids ]
print('Done')

In [ ]:
records

In [ ]:
# Write it into a fasta format file to use MUSCLE
SeqIO.write(records, 'homo_sapiens_dop1r1_similar.fna', 'fasta')

### **Step 5: Run MUSCLE to align multiple sequences**

In [ ]:
# Biopython can run MUSCLE from inside Python.

from Bio.Align.Applications import MuscleCommandline
muscle_app = 'MUSCLE/muscle3.8.31_i86linux64'  # Path to the binary file.
cline = MuscleCommandline(muscle_app, input="homo_sapiens_dop1r1_similar.fna", out="homo_sapiens_dop1r1_similar.txt")
print(cline)
output = cline()
print(output[1])  # print commandline output of the MUSCLE

### **Step 6: Check the results**

In [ ]:
# The multiple sequence alignemnt result is in the homo_sapiens_dop1r1_similar.txt

for seq_record in SeqIO.parse('homo_sapiens_dop1r1_similar.txt', 'fasta'):
    print(seq_record.id)
    print(seq_record.description)
    print('Length:', len(seq_record))
    print(seq_record.seq)
    print('---------------------------------')
    
    
# Now you can further analyze this data as you wish.

---

## **Endnote**

A real biologist may need to spend one or more days or even weeks studying Biopython to collect useful code and establish his or her own automated pipeline for the first time. That is, you will spend much time selecting a right package, finding the right code snippets, rewriting them to fit the need, and testing them. If you add your own analysis code that is not available out there (congrats! you're on the frontline of science), it may take even months to write just a few lines of code (remember the G-C skew plot that took many years of research to come up with a few lines of simple code?).


Now you have seen real-life examples of biological sequence analyses and code writing practice, I hope you can begin your own scientific exploration.
